In [25]:
import numpy as np
from scipy.spatial import Delaunay, KDTree
import math 

def preprocess_cores(cores):
    """
    Preprocess the core data to normalize positions, inverting the y-axis so that
    the numbering starts from the top left corner.
    :param cores: List of dictionaries with 'x', 'y', and 'radius' keys.
    :return: Normalized list of core coordinates with the y-axis inverted.
    """
    coordinates = np.array([(core['x'], core['y']) for core in cores])
    min_x, max_y = np.min(coordinates[:, 0]), np.max(coordinates[:, 1])
    normalized_inverted_coordinates = np.array([(x - min_x, max_y - y) for x, y in coordinates])
    return normalized_inverted_coordinates

import pandas as pd

labels_path = 'augmented_labels/158871_aug_6.json'

data = pd.read_json(labels_path)
cores = data.to_dict('records')

# Step 1: Perform Delaunay triangulation and get all edges
def get_all_edges_from_triangulation(triangulation):
    edges = set()
    # For each triangle, add edges to the set, avoiding duplicates
    for simplex in triangulation.simplices:
        for i in range(3):
            edge = tuple(sorted([simplex[i], simplex[(i + 1) % 3]]))
            edges.add(edge)
    return np.array(list(edges))


def calculate_edge_lengths(edges, coordinates):
    return np.array([np.linalg.norm(coordinates[start] - coordinates[end]) for start, end in edges])

def calculate_mad_bounds(edge_lengths, threshold_multiplier):
    median = np.median(edge_lengths)
    mad = np.median(np.abs(edge_lengths - median))
    lower_bound = median - (threshold_multiplier * mad)
    upper_bound = median + (threshold_multiplier * mad)
    return lower_bound, upper_bound

def filter_edges_within_bounds(edges, edge_lengths, lower_bound, upper_bound):
    return [edge for edge, length in zip(edges, edge_lengths) if length <= upper_bound]

def order_edges_to_point_right(filtered_edges, coordinates):
    return [(end, start) if coordinates[start][0] > coordinates[end][0] else (start, end) for start, end in filtered_edges]

def filter_edges_by_length(edges, coordinates, threshold_multiplier=1.5):
    edge_lengths = calculate_edge_lengths(edges, coordinates)
    lower_bound, upper_bound = calculate_mad_bounds(edge_lengths, threshold_multiplier)
    filtered_edges = filter_edges_within_bounds(edges, edge_lengths, lower_bound, upper_bound)
    ordered_edges = order_edges_to_point_right(filtered_edges, coordinates)
    return ordered_edges
def calculate_edge_angle(start_coord, end_coord):
    """
    Calculate the angle of an edge with respect to the x-axis.

    Parameters:
    start_coord (tuple): The starting coordinate of the edge.
    end_coord (tuple): The ending coordinate of the edge.

    Returns:
    float: The angle in degrees.
    """
    dx = end_coord[0] - start_coord[0]
    dy = end_coord[1] - start_coord[1]
    angle_radians = math.atan2(dy, dx)
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees

def filter_edges_by_angle(edges, coordinates, threshold_angle, origin_angle):
    """
    Filter edges based on their angle with respect to the x-axis.

    Parameters:
    edges (list of tuples): Indices of the edges.
    coordinates (dict): Coordinate values corresponding to the edge indices.
    threshold_angle (float): The +/- threshold angle from the origin angle.
    origin_angle (float): The angle around which the threshold is calculated.

    Returns:
    list: Filtered edges that fall within the specified angle range.
    """
    filtered_edges = []
    for start, end in edges:
        edge_angle = calculate_edge_angle(coordinates[start], coordinates[end])
        if origin_angle - threshold_angle <= edge_angle <= origin_angle + threshold_angle:
            filtered_edges.append((start, end))

    return filtered_edges


In [26]:



normalized_coordinates = preprocess_cores(cores)
triangulation = Delaunay(normalized_coordinates)
all_edges = get_all_edges_from_triangulation(triangulation)
length_filtered_edges = filter_edges_by_length(all_edges, normalized_coordinates, 1.5)
angle_filtered_edges = filter_edges_by_angle(length_filtered_edges, normalized_coordinates, 10, 30)


KeyboardInterrupt: 

In [33]:
length_filtered_edges

[(55, 57),
 (36, 33),
 (45, 33),
 (45, 44),
 (39, 26),
 (48, 46),
 (14, 11),
 (46, 36),
 (41, 42),
 (59, 55),
 (19, 18),
 (51, 41),
 (28, 18),
 (10, 9),
 (56, 48),
 (16, 15),
 (25, 15),
 (41, 30),
 (35, 37),
 (17, 62),
 (5, 60),
 (39, 38),
 (49, 48),
 (32, 21),
 (48, 38),
 (42, 31),
 (57, 49),
 (17, 16),
 (53, 45),
 (47, 50),
 (26, 16),
 (38, 25),
 (12, 13),
 (23, 22),
 (0, 2),
 (54, 46),
 (50, 49),
 (46, 45),
 (23, 13),
 (4, 54),
 (15, 12),
 (61, 51),
 (9, 62),
 (24, 23),
 (24, 12),
 (40, 29),
 (21, 14),
 (33, 23),
 (52, 60),
 (29, 19),
 (25, 24),
 (58, 55),
 (57, 56),
 (21, 20),
 (3, 5),
 (53, 52),
 (30, 20),
 (19, 10),
 (54, 53),
 (38, 36),
 (30, 31),
 (26, 25),
 (3, 53),
 (34, 32),
 (27, 26),
 (43, 32),
 (22, 21),
 (29, 35),
 (61, 43),
 (18, 17),
 (28, 27),
 (27, 17),
 (37, 27),
 (37, 39),
 (20, 11),
 (58, 47),
 (5, 52),
 (60, 61),
 (2, 1),
 (34, 22),
 (11, 1),
 (13, 0),
 (29, 28),
 (50, 37),
 (18, 9),
 (12, 6),
 (43, 41),
 (4, 3),
 (55, 50),
 (36, 24),
 (52, 61),
 (14, 2),
 (47, 3

In [29]:
calculate_edge_angle((239, 345), (190, 415))

124.99202019855868

In [35]:
len(angle_filtered_edges)

52